The task: Language modelling using a neural 4gram model. Basically the input to the neural model is past 4 words and what we try to predict is the 5th word in a sequence.  

The dataset used is wikitext, can be downloaded from [this link](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/)

In [2]:
import torch
from torch import nn, tensor
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
from tqdm import tqdm
import torch.optim as optim
from torch.nn import functional as F


In [4]:
class Data_set(Dataset):
  
  def __init__(self, path):
    lines=0
    

    with open(path, 'r') as f:
      for line in f:
        lines += 1

    ud_tags=[]
    tokens = []
    with open(path, 'r') as f:
      for line in tqdm(f, total=lines, desc="Building vocab"):
        words=line.split('\t')
        
        if line[0].isnumeric()==False:
          continue
        tokens += [[words[1].lower()]]
        ud_tags+=[words[3]]
    
        
    self.vocab = build_vocab_from_iterator(tokens, min_freq=1,
                                           specials=['<PAD>','<UNK>'],
                                           special_first=False)
    
    self.vocab.set_default_index(self.vocab.get_stoi()['<UNK>'])
    ud_tags+=["PAD_POS","UNK_TAG"]
    ud_tags=list(set(ud_tags))
    self.pos_tags={tag: idx for idx, tag in enumerate(ud_tags)}
    print(self.pos_tags)
    max_len=0
    with open(path, 'r') as f:
      all_sentences=[]
      all_tags=[]

      sen=[]
      tags=[]
      for line in tqdm(f, total=lines, desc="Generating data"):
        
        words=line.split('\t')
        
        if line[0].isnumeric()==False:
          if len(sen)!=0:
            all_sentences.append(sen)
            max_len=max(max_len,len(sen))
          if len(tags)!=0: all_tags.append(tags)
          sen=[]
          tags=[]
          continue
        sen += [words[1].lower()]
        tags+= [words[3]]
      # all_sentences.append(sen)
      # all_tags.append(tags)
      print(f"\nMax length of sentence {max_len}")
    self.max_len=max_len
    for i in range(len(all_sentences)):
      all_sentences[i]=[self.vocab[word] for word in all_sentences[i]]
      all_tags[i]=[self.pos_tags[tag] for tag in all_tags[i]]
      all_sentences[i]+=[self.vocab["<PAD>"]]*(max_len-len(all_sentences[i]))
      all_tags[i]+=[self.pos_tags["PAD_POS"]]*(max_len-len(all_tags[i]))

    # print(self.vocab.get_stoi())
    # print(self.pos_tags.get_stoi())
    
    self.all_sentences=tensor(all_sentences)
    self.all_tags=tensor(all_tags)
    print(len(self.all_sentences))
    # print(self.all_sentences[-1])

    # for i in range(len(all_sentences)):
    #   print(f"{all_sentences[i]}\n{all_tags[i]}\n\n")

  def __len__(self) -> int:
      return self.all_sentences.shape[0]
      
  def __getitem__(self, index : int): # -> tuple[tensor, tensor]
      return self.all_sentences[index], self.all_tags[index] # source, target

    

ds_obj=Data_set('en_atis-ud-train.conllu')
torch.save(ds_obj,'ds_obj.pt')
ds_obj=torch.load('ds_obj.pt')
# ds_obj=Data_set('sample.txt')

  
  

Building vocab: 100%|██████████| 61477/61477 [00:00<00:00, 415403.87it/s]


{'DET': 0, 'NOUN': 1, 'VERB': 2, 'AUX': 3, 'INTJ': 4, 'ADP': 5, 'CCONJ': 6, 'PART': 7, 'NUM': 8, 'PROPN': 9, 'ADV': 10, 'PAD_POS': 11, 'UNK_TAG': 12, 'PRON': 13, 'ADJ': 14}


Generating data: 100%|██████████| 61477/61477 [00:00<00:00, 840490.46it/s]



Max length of sentence 46
4274


## Dataloader

- used to load data from the dataset in batches

In [6]:

train_dataloader = DataLoader(ds_obj, batch_size=64, shuffle=True)
print(len(train_dataloader))

67


In [7]:
features, labels = next(iter(train_dataloader))
print(f"Feature batch shape: {features.size()}")
print(f"Labels batch shape: {labels.size()}")

Feature batch shape: torch.Size([64, 46])
Labels batch shape: torch.Size([64, 46])


# The model
- init: The architecture
- Forward: computing the output given an input. y = Model(x)

In [8]:
# To use GPU 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [9]:
class LSTMModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim,hidden_size, output_dim):
    super(LSTMModel, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.lstm=nn.LSTM(embedding_dim,hidden_size,1,batch_first=True)#output would be Batchsize x max_len_sen x hidden_size
    self.fc = nn.Linear(hidden_size, output_dim)
    self.activ = nn.ReLU()

  def __len__(self) -> int:
      return self.all_sentences.shape[0]
      
  def __getitem__(self, index : int): # -> tuple[tensor, tensor]
      return self.all_sentences[index], self.all_tags[index] # source, target



  def forward(self, inputs):
                    # [bz,len_sen]
    # print(f"Inputs size {inputs.size()}")
    embeds = self.embeddings(inputs)
    # [bz, len_sen, 10]
    # print(f"Embeds size {embeds.size()}")
    out,_=self.lstm(embeds)
    out = self.fc(out)
    # [bz, len_sen, vocab_size]
    # print(f"Output shape {out.shape}")
    return out

# The training loop 
- Set hyperparameters
- Pick loss function and optimizer
- Make train, validation/test loops


In [10]:
# Hyperparameters
vocab_size = len(ds_obj.vocab)
print(vocab_size)
embedding_dim = 200
model =LSTMModel(vocab_size, embedding_dim,150,len(ds_obj.pos_tags))
epochs = 6
learning_rate = 0.01
# batch size is also a hyperparameter
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

865


In [19]:
from itertools import chain
from sklearn.metrics import classification_report
import numpy
target_names=[]
for i in range(0,len(ds_obj.pos_tags)):
  for tag,idx in ds_obj.pos_tags.items():
    if idx==i:
      target_names.append(tag)

def get_whole_data(path,ds_obj):
  with open(path, 'r') as f:
    all_sentences=[]
    all_tags=[]
    max_len=0
    sen=[]
    tags=[]
    for line in f.readlines():
      
      words=line.split('\t')
      
      if line[0].isnumeric()==False:
        if len(sen)!=0:
          all_sentences.append(sen)
          max_len=max(max_len,len(sen))
        if len(tags)!=0: all_tags.append(tags)
        sen=[]
        tags=[]
        continue
      sen += [words[1].lower()]
      tags+= [words[3]]
    all_sentences.append(sen)
    all_tags.append(tags)
    print(f"\nMax length of sentence {max_len}")
    for i in range(len(all_sentences)):
      all_sentences[i]=[ds_obj.vocab[word] for word in all_sentences[i]]
      all_tags[i]=[ds_obj.pos_tags.get(tag,ds_obj.pos_tags["UNK_TAG"]) for tag in all_tags[i]]

    return all_sentences, all_tags,max_len

def get_x_and_y(path,ds_obj,batch_size):
  all_sentences,all_tags,max_len=get_whole_data(path,ds_obj)
  
  # print(all_sentences)
  
  for i in range(len(all_sentences)):
      all_sentences[i]+=[ds_obj.vocab["<PAD>"]]*(max_len-len(all_sentences[i]))
      all_tags[i]+=[ds_obj.pos_tags["PAD_POS"]]*(max_len-len(all_tags[i]))
  print(f"Len of all sentences: {len(all_sentences)}")
  comb_data=[]
  for i in range(0, len(all_sentences), batch_size):
    h=min(i+batch_size,len(all_sentences))
    comb_data.append((all_sentences[i:h],all_tags[i:h]))
  print(f"Num of batches {len(comb_data)}")
  
  return comb_data
  

val_data = get_x_and_y('en_atis-ud-dev.conllu',ds_obj,64)
# for i,(x_val,y_val) in enumerate(val_data):
#   print(len(x_val),len(y_val[0]))


Max length of sentence 42
Len of all sentences: 573
Num of batches 9


In [12]:
print(target_names)
print(ds_obj.pos_tags)

['DET', 'NOUN', 'VERB', 'AUX', 'INTJ', 'ADP', 'CCONJ', 'PART', 'NUM', 'PROPN', 'ADV', 'PAD_POS', 'UNK_TAG', 'PRON', 'ADJ']
{'DET': 0, 'NOUN': 1, 'VERB': 2, 'AUX': 3, 'INTJ': 4, 'ADP': 5, 'CCONJ': 6, 'PART': 7, 'NUM': 8, 'PROPN': 9, 'ADV': 10, 'PAD_POS': 11, 'UNK_TAG': 12, 'PRON': 13, 'ADJ': 14}


In [20]:
# The main loop 

print("Starting Training...")
e_num=1
a=torch.nn.Softmax(dim=1)
for epoch in range(epochs):
  # Training loop
  train_loss = 0
  model.train()
  num_batch=0
  all_pred=[]
  all_actual=[]

  for batch in tqdm(train_dataloader, desc="Training"):
    x_train, y_train = batch
    optimizer.zero_grad()# clears the gradients from prev iteration 
    # print('train',x_train.shape)
    output = model(x_train) # forward pass
    output=output.view(-1,output.shape[2])
    output = a(output)

    # print('test',output.shape,y_train.shape)
    loss = loss_fn(output, y_train.view(-1)) # calculate loss

    #backprop
    loss.backward() # compute gradients
    optimizer.step() # update parameters
    
    train_loss += loss.item()
    num_batch+=1

    val_loss = 0


  # Validation loop 
  val_loss = 0
  model.eval()
  for i, (x_val, y_val) in enumerate(val_data):
    output = model(tensor(x_val))
    output=output.view(-1,output.shape[2])
    output=a(output)
    all_pred+=[torch.argmax(output[i]) for i in range(len(output))]
    all_actual+=list(chain.from_iterable(y_val))
    loss = loss_fn(output, tensor(list(chain.from_iterable(y_val))))
    val_loss += loss.item()
  print(classification_report(all_actual, all_pred, target_names=target_names))
  print(f'Epoch {e_num} \t\t Training Loss: {train_loss / len(train_dataloader)} \t\t Validation Loss: {val_loss / len(val_data)}')
  e_num+=1
    

  # print(train_loss/len(train_dataloader))
  # print(f"Number of batches {num_batch}")

  


Starting Training...


Training: 100%|██████████| 67/67 [00:20<00:00,  3.20it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         DET       0.99      0.87      0.93       568
        NOUN       0.98      0.98      0.98      1143
        VERB       1.00      0.97      0.98       653
         AUX       0.96      0.99      0.97       266
        INTJ       1.00      1.00      1.00        35
         ADP       0.94      1.00      0.97      1415
       CCONJ       1.00      0.99      1.00       107
        PART       0.00      0.00      0.00        73
         NUM       0.98      0.97      0.98       131
       PROPN       0.98      1.00      0.99      1551
         ADV       0.96      0.78      0.86        59
     PAD_POS       1.00      1.00      1.00     17422
     UNK_TAG       0.00      0.00      0.00         2
        PRON       0.88      0.99      0.93       414
         ADJ       0.95      0.93      0.94       227

    accuracy                           0.99     24066
   macro avg       0.84      0.83      0.83     24066
weighted avg       0.99   

Training: 100%|██████████| 67/67 [00:21<00:00,  3.12it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         DET       0.99      0.87      0.93       568
        NOUN       0.98      0.98      0.98      1143
        VERB       1.00      0.97      0.98       653
         AUX       0.96      0.99      0.97       266
        INTJ       1.00      1.00      1.00        35
         ADP       0.94      1.00      0.97      1415
       CCONJ       1.00      0.99      1.00       107
        PART       0.00      0.00      0.00        73
         NUM       0.98      0.97      0.98       131
       PROPN       0.98      1.00      0.99      1551
         ADV       0.96      0.78      0.86        59
     PAD_POS       1.00      1.00      1.00     17422
     UNK_TAG       0.00      0.00      0.00         2
        PRON       0.88      0.99      0.93       414
         ADJ       0.95      0.93      0.94       227

    accuracy                           0.99     24066
   macro avg       0.84      0.83      0.83     24066
weighted avg       0.99   

...

Training: 100%|██████████| 67/67 [00:21<00:00,  3.14it/s]
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         DET       0.99      0.87      0.93       568
        NOUN       0.98      0.98      0.98      1143
        VERB       1.00      0.97      0.98       653
         AUX       0.96      0.99      0.97       266
        INTJ       1.00      1.00      1.00        35
         ADP       0.94      1.00      0.97      1415
       CCONJ       1.00      0.99      1.00       107
        PART       0.00      0.00      0.00        73
         NUM       0.98      0.97      0.98       131
       PROPN       0.98      1.00      0.99      1551
         ADV       0.96      0.78      0.86        59
     PAD_POS       1.00      1.00      1.00     17422
     UNK_TAG       0.00      0.00      0.00         2
        PRON       0.88      0.99      0.93       414
         ADJ       0.95      0.93      0.94       227

    accuracy                           0.99     24066
   macro avg       0.84      0.83      0.83     24066
weighted avg       0.99   

Training: 100%|██████████| 67/67 [00:20<00:00,  3.22it/s]


              precision    recall  f1-score   support

         DET       0.99      0.87      0.93       568
        NOUN       0.98      0.98      0.98      1143
        VERB       1.00      0.97      0.98       653
         AUX       0.96      0.99      0.97       266
        INTJ       1.00      1.00      1.00        35
         ADP       0.94      1.00      0.97      1415
       CCONJ       1.00      0.99      1.00       107
        PART       0.00      0.00      0.00        73
         NUM       0.98      0.97      0.98       131
       PROPN       0.98      1.00      0.99      1551
         ADV       0.96      0.78      0.86        59
     PAD_POS       1.00      1.00      1.00     17422
     UNK_TAG       0.00      0.00      0.00         2
        PRON       0.88      0.99      0.93       414
         ADJ       0.95      0.93      0.94       227

    accuracy                           0.99     24066
   macro avg       0.84      0.83      0.83     24066
weighted avg       0.99   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
torch.save(model, 'trained_model.pt')

In [23]:
model = torch.load('trained_model.pt')
x=input("Enter Sentence: ")
words=x.lower().split(' ')
x=tensor([ds_obj.vocab[word] for word in words])#replacing words by indices
output=model(x)
output=a(output)
print(output.shape)
output=[torch.argmax(output[i]) for i in range(len(words))]
for i in range(len(words)):
  for tag,idx in ds_obj.pos_tags.items():
    if idx==output[i]:
      print(f"{words[i]}\t {tag}")
      break
  
print(output)

Enter Sentence: what are flights between baltimore and dallas
torch.Size([7, 15])
what	 PRON
are	 AUX
flights	 NOUN
between	 ADP
baltimore	 PROPN
and	 CCONJ
dallas	 PROPN
[tensor(13), tensor(3), tensor(1), tensor(5), tensor(9), tensor(6), tensor(9)]


In [ ]:
print(ds_obj.pos_tags)

{'ADP': 0, 'PRON': 1, 'PROPN': 2, 'CCONJ': 3, 'AUX': 4, 'DET': 5, 'ADJ': 6, 'INTJ': 7, 'PART': 8, 'PAD_POS': 9, 'UNK_TAG': 10, 'NOUN': 11, 'NUM': 12, 'VERB': 13, 'ADV': 14}


In [30]:
test_dataset = get_x_and_y('en_atis-ud-test.conllu',ds_obj,64)
# target_names.remove('UNK_TAG')
val_loss = 0
all_pred=[]
all_actual=[]
model.eval()
for i, (x_val, y_val) in enumerate(test_dataset):
  output = model(tensor(x_val))
  output=output.view(-1,output.shape[2])
  all_pred+=[torch.argmax(output[i]) for i in range(len(output))]
  all_actual+=list(chain.from_iterable(y_val))
  
  loss = loss_fn(output, tensor(list(chain.from_iterable(y_val))))
  val_loss += loss.item()

print(val_loss/len(test_dataset))
print(classification_report(all_actual, all_pred, target_names=target_names))


Max length of sentence 33
Len of all sentences: 587
Num of batches 10
0.1101912021636963
              precision    recall  f1-score   support

         DET       0.98      0.87      0.92       512
        NOUN       1.00      0.99      0.99      1166
        VERB       0.99      0.95      0.97       629
         AUX       0.92      0.99      0.95       256
        INTJ       1.00      1.00      1.00        36
         ADP       0.96      1.00      0.98      1434
       CCONJ       1.00      0.98      0.99       109
        PART       0.00      0.00      0.00        56
         NUM       0.94      0.95      0.95       127
       PROPN       0.98      1.00      0.99      1567
         ADV       1.00      0.66      0.79        76
     PAD_POS       1.00      1.00      1.00     12791
        PRON       0.86      0.97      0.91       392
         ADJ       0.92      0.98      0.95       220

    accuracy                           0.99     19371
   macro avg       0.90      0.88      0.89 

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
